In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///:memory:')

Session = sessionmaker(bind=engine)

session = Session()

In [2]:
from datetime import datetime

from sqlalchemy import Column, Integer, Numeric, String, DateTime, ForeignKey, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref


Base = declarative_base()


class Cookie(Base):
    __tablename__ = 'cookies'
    
    cookie_id = Column(Integer(), primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_recipe_url=Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer())
    unit_cost=Column(Numeric(12, 2))
    
    def __init__(self, name, recipe_url=None, sku=None, quantity=0, unit_cost=0.00):
        self.cookie_name = name
        self.cookie_recipe_url = recipe_url
        self.cookie_sku = sku
        self.quantity = quantity
        self.unit_cost = unit_cost
        
    def __repr__(self):
        return "Cookie(cookie_name='{self.cookie_name}', " \
                       "cookie_recipe_url='{self.cookie_recipe_url}', " \
                       "cookie_sku='{self.cookie_sku}', " \
                       "qunatity={self.quantity}, " \
                       "unit_cost={self.unit_cost})".format(self=self)
    
    
class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key=True)
    username = Column(String(15), nullable=False, unique=True)
    email_address = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=False)
    password = Column(String(25), nullable=False)
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now, onupdate=datetime.now)
    
    def __init__(self, username, email_address, phone, password):
        self.username = username
        self.email_address = email_address
        self.phone = phone
        self.password = password
    
    def __repr__(self):
        return "User(username='{self.username}', " \
                     "email_address='{self.email_address}', " \
                     "phone='{self.phone}', " \
                     "password='{self.password}')".format(self=self)
    
    
class Order(Base):
        __tablename__ = 'orders'
        order_id = Column(Integer(), primary_key=True)
        user_id = Column(Integer(), ForeignKey('users.user_id'))
        shipped = Column(Boolean(), default=False)
        
        user = relationship("User", backref=backref('orders', order_by=order_id))
        
        def __repr__(self):
            return "Order(user_id={self.user_id}, " \
                          "shipped={self.shipped})".format(self=self)
        
        
class LineItem(Base):
    __tablename__ = 'line_items'
    line_item_id = Column(Integer(), primary_key=True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    cookie_id = Column(Integer(), ForeignKey('cookies.cookie_id'))
    quantity = Column(Integer())
    extended_cost = Column(Numeric(12, 2))
    
    order = relationship("Order", backref=backref('line_items', order_by=line_item_id))
    cookie = relationship("Cookie", uselist=False)
    
    def __repr__(self):
        return "LineItems(order_id={self.order_id}, " \
                          "cookie_id={self.cookie_id}, " \
                          "quantity={self.quantity}, " \
                          "extended_cost={self.extended_cost})".format(self=self)
    
    
Base.metadata.create_all(engine)

In [3]:
cc_cookie = Cookie('chocolate chip',
                   'http:some.aweso.me/cookie/recipe.html',
                   'CC01', 12, 0.50)

In [4]:
from sqlalchemy import inspect
insp = inspect(cc_cookie)

In [5]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

 transient: True
   pending: False
persistent: False
  detached: False


In [6]:
session.add(cc_cookie)

In [7]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

 transient: False
   pending: True
persistent: False
  detached: False


In [8]:
session.expunge(cc_cookie)

In [9]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

 transient: True
   pending: False
persistent: False
  detached: False


In [10]:
session.add(cc_cookie)
cc_cookie.cookie_name = 'Change chocolate chip'

In [11]:
insp.modified

True

In [12]:
for attr, attr_state in insp.attrs.items():
    if attr_state.history.has_changes():
        print('{}: {}'.format(attr, attr_state.value))
        print('History: {}\n'.format(attr_state.history))

cookie_name: Change chocolate chip
History: History(added=['Change chocolate chip'], unchanged=(), deleted=())

cookie_recipe_url: http:some.aweso.me/cookie/recipe.html
History: History(added=['http:some.aweso.me/cookie/recipe.html'], unchanged=(), deleted=())

cookie_sku: CC01
History: History(added=['CC01'], unchanged=(), deleted=())

quantity: 12
History: History(added=[12], unchanged=(), deleted=())

unit_cost: 0.5
History: History(added=[0.5], unchanged=(), deleted=())



In [13]:
dcc = Cookie('dark chocolate chip',
             'http://some.aweso.me/cookie/recipe_dark.html',
             'CC02', 1, 0.75)
session.add(dcc)
session.commit()

In [14]:
results = session.query(Cookie).one()

C:\Users\Mathew Holden\Anaconda3\lib\site-packages\sqlalchemy\sql\sqltypes.py:603: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


MultipleResultsFound: Multiple rows were found for one()

In [15]:
from sqlalchemy.orm.exc import MultipleResultsFound
try:
    results = session.query(Cookie).one()
except MultipleResultsFound as exc:
    print("We found too many cookies...is that even possible")

We found too many cookies...is that even possible


In [16]:
session.query(Cookie).all()

[Cookie(cookie_name='Change chocolate chip', cookie_recipe_url='http:some.aweso.me/cookie/recipe.html', cookie_sku='CC01', qunatity=12, unit_cost=0.50),
 Cookie(cookie_name='dark chocolate chip', cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html', cookie_sku='CC02', qunatity=1, unit_cost=0.75)]

In [17]:
cookiemon = User('cookiemon', 'mon@cookie.com', '111-111-1111', 'password')
session.add(cookiemon)
o1 = Order()
o1.user = cookiemon
session.add(o1)

cc = session.query(Cookie).filter(Cookie.cookie_name == "Change chocolate chip").one()

line1 = LineItem(order=o1, cookie=cc, quantity=2, extended_cost=1.00)

session.add(line1)
session.commit()

In [18]:
order = session.query(Order).first()
session.expunge(order)
order.line_items.all()

DetachedInstanceError: Parent instance <Order at 0x1cb7cefaa58> is not bound to a Session; lazy load operation of attribute 'line_items' cannot proceed (Background on this error at: http://sqlalche.me/e/bhk3)

In [19]:
cookiemon = User('cookiemon', 'mon@cookie.com', '111-111-1111', 'password')
session.add(cookiemon)
o1 = Order()
o1.user = cookiemon
session.add(o1)

cc = session.query(Cookie).filter(Cookie.cookie_name == "Change chocolate chip").one()

line1 = LineItem(order=o1, cookie=cc, quantity=2, extended_cost=1.00)

session.add(line1)
session.commit()

IntegrityError: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely) (sqlite3.IntegrityError) UNIQUE constraint failed: users.username [SQL: 'INSERT INTO users (username, email_address, phone, password, created_on, updated_on) VALUES (?, ?, ?, ?, ?, ?)'] [parameters: ('cookiemon', 'mon@cookie.com', '111-111-1111', 'password', '2018-12-04 20:09:50.756302', '2018-12-04 20:09:50.756302')] (Background on this error at: http://sqlalche.me/e/gkpj)

In [20]:
cookies = session.query(Cookie).all()

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely) (sqlite3.IntegrityError) UNIQUE constraint failed: users.username [SQL: 'INSERT INTO users (username, email_address, phone, password, created_on, updated_on) VALUES (?, ?, ?, ?, ?, ?)'] [parameters: ('cookiemon', 'mon@cookie.com', '111-111-1111', 'password', '2018-12-04 20:09:50.756302', '2018-12-04 20:09:50.756302')] (Background on this error at: http://sqlalche.me/e/gkpj)

In [21]:
session.rollback()

In [22]:
from sqlalchemy.exc import IntegrityError
try:
    cookiemon = User('cookiemon', 'mon@cookie.com', '111-111-1111', 'password')
    session.add(cookiemon)
    o1 = Order()
    o1.user = cookiemon
    session.add(o1)
    
    cc = session.query(Cookie).filter(Cookie.cookie_name ==
                                   "Change chocolate chip").one()
    line1 = LineItem(order=o1, cookie=cc, quantity=2, extended_cost=1.00)
    
    session.add(line1)
    session.commit()
except IntegrityError as err:
    print('ERROR: {}'.format(err.orig))
    session.rollback()

ERROR: UNIQUE constraint failed: users.username


In [23]:
session.query(Order).all()

[Order(user_id=1, shipped=False)]

In [26]:
print("Critical Thinking Section: In your own words, explain in a paragraph (5 sentences or more) describing what is a data integrity error.")

Critical Thinking Section: In your own words, explain in a paragraph (5 sentences or more) describing what is a data integrity error.


In [3]:
print("Data integrity refers to the acuracy and consistency of data. ")
print("The data structure from a page that does not match the data structure in the database will cause a Data Integrity Error. Maintaining data consistent throughout its lifecycle is import in protecting it. Although something as small as unaccurate data type may seem small, it can make a big impact.")


Data integrity refers to the acuracy and consistency of data. 
The data structure from a page that does not match the data structure in the database will cause a Data Integrity Error. Maintaining data consistent throughout its lifecycle is import in protecting it. Although something as small as unaccurate data type may seem small, it can make a big impact.


In [28]:
print("Mathew Holden")

Mathew Holden
